In [8]:
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

### Get Data ###

In [9]:
df_image = pd.read_csv('label_and_path.csv')
X_image = df_image[:10000]['image_path'].values

df_meta = pd.read_csv('skin_data.csv')
X_meta = df_meta[:10000].drop(columns=['target', 'image_path']).values

y = df_image[:10000]['label'].values

### Split Data ###

In [10]:
# Split for images
X_train_image, X_val_test_image, y_train_image, y_val_test_image = train_test_split(X_image, y, test_size=0.3)
X_val_image, X_test_image, y_val_image, y_test_image = train_test_split(X_val_test_image, y_val_test_image, test_size=0.5)

# Split for metadata
X_train_meta, X_val_test_meta, y_train_meta, y_val_test_meta = train_test_split(X_meta, y, test_size=0.3)
X_val_meta, X_test_meta, y_val_meta, y_test_meta = train_test_split(X_val_test_meta, y_val_test_meta, test_size=0.5)

In [11]:
def preprocess_combined(image_path, metadata, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    return (image, metadata), label

def create_combined_dataset(image_paths, metadata, labels, batch_size=32, shuffle=True):
    image_paths = tf.constant(image_paths)
    metadata = tf.convert_to_tensor(metadata, dtype=tf.float32)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)

    ds = tf.data.Dataset.from_tensor_slices((image_paths, metadata, labels))
    ds = ds.map(preprocess_combined, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=1000)

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [12]:
train_ds = create_combined_dataset(X_train_image, X_train_meta, y_train_image, batch_size=256)
val_ds = create_combined_dataset(X_val_image, X_val_meta, y_val_image, batch_size=256, shuffle=False)
test_ds = create_combined_dataset(X_test_image, X_test_meta, y_test_image, batch_size=256, shuffle=False)

### CNN + MLP ###

In [13]:

# CNN Model
def CNN_model(input_shape=(128, 128, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Optional: fine-tune later
    
    x = MaxPooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    return Model(inputs=base_model.input, outputs=x, name='cnn_model')


# MLP Model
def MLP_model(input_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(df_meta.shape[1], activation='relu')(x)
    
    return models.Model(inputs=inputs, outputs=x, name='metadata_model')


# Combined Model
def build_combined_model(image_shape, metadata_dim):
    cnn_model = CNN_model(image_shape)
    mlp_model = MLP_model(metadata_dim)

    # Flatten the CNN output to make it 2D
    cnn_flattened = layers.Flatten()(cnn_model.output)

    # Fusion
    combined = layers.concatenate([cnn_flattened, mlp_model.output])
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='sigmoid')(x)  # Connect the output layer to `x`

    model = models.Model(inputs=[cnn_model.input, mlp_model.input], outputs=output)
    return model

In [21]:
model = build_combined_model(image_shape=(128, 128, 3), metadata_dim=X_train_meta.shape[1])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=1,
    callbacks=[early_stopping_callback]
)

28/28 ━━━━━━━━━━━━━━━━━━━━ 127s 4s/step - accuracy: 0.9882 - loss: 0.3382 - val_accuracy: 0.9980 - val_loss: 0.3336


In [22]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.9994 - loss: 0.1187 
Test Loss: 0.26201409101486206
Test Accuracy: 0.9986666440963745


### Auto Encoder ###

# Define the convolutional autoencoder architecture
input_shape = X_train.shape[1:]

# Input layer
input_layer = Input(shape=input_shape)

# Encoding layers
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoding layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Encoder model
encoder = Model(input_layer, encoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=1, batch_size=256, shuffle=True, validation_data=(X_val, X_val), callback=[early_stopping_callback])

# Encode the validation data
encoded_val = encoder.predict(X_val)

# Detect anomalies using reconstruction loss
reconstructed_val = autoencoder.predict(X_val)
reconstruction_error = np.mean((X_val - reconstructed_val) ** 2, axis=(1, 2, 3))

# Set a threshold for anomaly detection
threshold = reconstruction_error.mean() + 2 * reconstruction_error.std()

# Identify anomalies
anomalies = reconstruction_error > threshold
print("Number of anomalies detected:", anomalies.sum())